# *Resultados Fundamentais - Extração de Dados Financeiros e de Empresas*
*Este script extrai dados financeiros de uma tabela na página de resultados do site "[Fundamentus](https://www.fundamentus.com.br)" e informações sobre papeis e empresas de outra tabela na página de detalhes do mesmo site.*

## *Instalação das Bibliotecas Necessárias*

In [29]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## *Importação das Bibliotecas Necessárias*

In [5]:
import os
import zipfile
import pandas as pd
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
)
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import logging
from io import StringIO

### Configuração de Logs

In [2]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

##  *Configuração e Inicialização do WebDriver do Selenium*

In [3]:
# Configuração e Inicialização do WebDriver do Selenium em Modo Headless
def configurar_driver_headless():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    return driver

driver = configurar_driver_headless()

2024-07-29 20:13:04,106 - INFO - ====== WebDriver manager ======
2024-07-29 20:13:09,211 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:13:10,221 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:13:11,149 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:13:13,606 - INFO - WebDriver version 126.0.6478.182 selected
2024-07-29 20:13:13,623 - INFO - Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.182/win32/chromedriver-win32.zip
2024-07-29 20:13:13,624 - INFO - About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/126.0.6478.182/win32/chromedriver-win32.zip
2024-07-29 20:13:14,769 - INFO - Driver downloading response is 200
2024-07-29 20:13:15,322 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:13:16,972 - INFO - Driver has been saved in cache [C:\Users\jvcca\.wdm\drivers\chromedriver\win64\126.0.6478.182]


## *URLs das Páginas a Serem Acessadas*

In [4]:
url_pagina1 = "https://www.fundamentus.com.br/resultado.php"
url_pagina2 = "https://www.fundamentus.com.br/detalhes.php"

## *Localização e Extração das Tabelas* 

In [5]:
def obter_tabela_html(driver, url, xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath))
    )
    elemento = driver.find_element("xpath", xpath)
    return elemento.get_attribute("outerHTML")

html_tabela = obter_tabela_html(driver, url_pagina1, "/html/body/div[1]/div[2]/table")
html_tabela_2 = obter_tabela_html(driver, url_pagina2, "/html/body/div[1]/div[2]/div[1]/div/table")

## Conversão em DataFrame Pandas

In [6]:
df_financas = pd.read_html(StringIO(html_tabela), decimal=",", thousands=".")[0]
df_empresas = pd.read_html(StringIO(html_tabela_2))[0]

# Extraindo o Papel e o Nome da Empresa, e Removendo a Coluna de Razão Social
df_empresas.columns = ["Papel", "Nome", "Razão Social"]
df_empresas.drop(columns=["Razão Social"], inplace=True)

## Exibindo os DataFrames

In [7]:
#Tabela com os dados financeiros
print("Tabela com os dados financeiros:")
display(df_financas)

Tabela com os dados financeiros:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,10.17,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"8,66%","5,65%",1.08,"15,25%","19,93%",0.0,5.458030e+08,0.82,"30,93%"
1,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
2,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
3,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.0,-9.105000e+06,-6.52,"-41,11%"
4,CLAN3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-1,05%",0.0,1.012240e+09,0.00,"-63,96%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,CSNA3,11.92,854.94,0.91,0.361,"12,24%",0.172,2.15,2.62,-0.40,...,4.82,"13,77%","1,70%",1.29,"8,82%","0,11%",83197300.0,1.729740e+10,2.68,"14,14%"
983,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
984,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
985,CEPE3,128.00,1910.07,5.88,1.143,"0,00%",0.691,37.60,8.14,-1.10,...,10.86,"14,04%","0,06%",1.08,"9,34%","0,31%",0.0,1.624000e+09,5.20,"1,81%"


In [8]:
#Tabela com as empresas e seus papeis
print("\nTabela com os papeis e seus nomes:")
display(df_empresas)


Tabela com os papeis e seus nomes:


,Papel,Nome
0,AALR3,ALLIAR
1,ABCB3,ABC Brasil
2,ABCB4,ABC Brasil
3,ABEV3,AMBEV S/A
4,ABRE3,SOMOS EDUCA
...,...,...
1029,WMBY3,WEMBLEY SOCIEDADE ANÔNIMA
1030,WSON33,Wilson Sons
1031,YBRA4,OPPORTUNITY ALEF S/A
1032,YDUQ3,YDUQS PART


## Fechando o Driver Selenium

In [9]:
# Fechar o WebDriver do Selenium após concluir a extração e exibição dos dados
driver.quit()

## Balanços

###  *Configuração e Inicialização do WebDriver do Selenium*

In [6]:
# Configuração das pastas
DOWNLOADS_FOLDER = "downloads"
BALANCOS_FOLDER = "balancos"
SEM_BALANCOS_LISTA = "sem_balancos.txt"

def criar_pasta_se_nao_existir(caminho_pasta):
    if not os.path.exists(caminho_pasta):
        os.makedirs(caminho_pasta)

criar_pasta_se_nao_existir(DOWNLOADS_FOLDER)
criar_pasta_se_nao_existir(BALANCOS_FOLDER)

In [29]:
# Configuração e Inicialização do WebDriver do Selenium
def configurar_driver():
    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": os.path.join(os.getcwd(), DOWNLOADS_FOLDER),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    return driver

driver = configurar_driver()

2024-07-29 20:44:11,876 - INFO - ====== WebDriver manager ======
2024-07-29 20:44:18,594 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:44:19,436 - INFO - Get LATEST chromedriver version for google-chrome
2024-07-29 20:44:20,483 - INFO - Driver [C:\Users\jvcca\.wdm\drivers\chromedriver\win64\126.0.6478.182\chromedriver-win32/chromedriver.exe] found in cache


### Função para Baixar e Renomear Arquivo

#### DataFrame de teste

In [ ]:
dados_empresas = {
    'Papel': [
        'AALR3', 'ABCB3', 'ABCB4', 'ABEV3', 'ABRE3', 'ABYA3',
        'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3',
        'AERI3', 'AESB3', 'AESL3', 'AESL4', 'AFLT3', 'AFLU3',
        'AFLU5', 'AGEI3'
    ]
}

df_empresas = pd.DataFrame(dados_empresas)
display(df_empresas)

#### Baixa e renomeia arquivos para todos os papéis na lista fornecida e registra erros.

In [31]:
def baixar_e_renomear_arquivo(df_empresas, driver):
    sem_balanco = set()  # Usar um conjunto para evitar duplicatas

    for papel in df_empresas['Papel']:
        arquivo_destino = os.path.join(DOWNLOADS_FOLDER, f"{papel}.zip")
        
        if os.path.isfile(arquivo_destino):
            logging.info(f"Arquivo {arquivo_destino} já existe. Pulando o papel {papel}.")
            continue

        url_papeis = f'https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile'
        driver.get(url_papeis)
        
        try:
            botao_download = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "a.bt-baixar"))
            )
            botao_download.click()
            
            WebDriverWait(driver, 30).until(
                lambda driver: any(f.startswith("bal_") for f in os.listdir(DOWNLOADS_FOLDER))
            )
            
            arquivos_iniciais = {f for f in os.listdir(DOWNLOADS_FOLDER)}
            novo_arquivo = next((f for f in os.listdir(DOWNLOADS_FOLDER) if f.startswith("bal_")), None)
            
            if novo_arquivo:
                caminho_arquivo = os.path.join(DOWNLOADS_FOLDER, novo_arquivo)
                try:
                    os.rename(caminho_arquivo, arquivo_destino)
                    logging.info(f"Arquivo {arquivo_destino} baixado e renomeado com sucesso!")
                except Exception as e:
                    logging.error(f"Erro ao renomear arquivo {novo_arquivo} para {papel}: {e}")
            else:
                logging.error(f"Erro ao encontrar o arquivo baixado para {papel}")
        
        except (NoSuchElementException, TimeoutException):
            logging.error(f"Botão de download não encontrado para {papel}.")
            sem_balanco.add(papel)  # Adicionar ao conjunto
            continue
        except Exception as e:
            logging.error(f"Erro inesperado para {papel}: {e}")
            continue

    with open(SEM_BALANCOS_LISTA, 'w') as f:
        for papel in sorted(sem_balanco):
            f.write(f"{papel}\n")

#### Chamando a função para baixar e renomear os arquivos

In [33]:
baixar_e_renomear_arquivo(df_empresas, driver)

2024-07-29 20:45:44,612 - INFO - Arquivo downloads\AALR3.zip baixado e renomeado com sucesso!
2024-07-29 20:45:49,806 - INFO - Arquivo downloads\ABCB3.zip baixado e renomeado com sucesso!
2024-07-29 20:45:54,311 - INFO - Arquivo downloads\ABCB4.zip baixado e renomeado com sucesso!
2024-07-29 20:46:00,630 - INFO - Arquivo downloads\ABEV3.zip baixado e renomeado com sucesso!
2024-07-29 20:46:12,610 - ERROR - Botão de download não encontrado para ABRE3.
2024-07-29 20:46:19,408 - INFO - Arquivo downloads\ABYA3.zip baixado e renomeado com sucesso!
2024-07-29 20:46:23,945 - INFO - Arquivo downloads\ACES3.zip baixado e renomeado com sucesso!
2024-07-29 20:46:29,203 - INFO - Arquivo downloads\ACES4.zip baixado e renomeado com sucesso!
2024-07-29 20:46:32,869 - INFO - Arquivo downloads\ADHM3.zip baixado e renomeado com sucesso!
2024-07-29 20:46:38,807 - INFO - Arquivo downloads\AEDU11.zip baixado e renomeado com sucesso!
2024-07-29 20:46:43,274 - INFO - Arquivo downloads\AEDU3.zip baixado e ren

In [34]:
# Fechar o WebDriver do Selenium após concluir a extração e exibição dos dados
driver.quit()

### Função para Extrair e Renomear Arquivo

In [17]:
def extrair_e_renomear_arquivo(df_empresas, download_path, balancos_path, sem_balancos_lista):
    # Ler lista de papéis sem balanço
    sem_balanco = []
    if os.path.exists(sem_balancos_lista):
        with open(sem_balancos_lista, 'r') as f:
            sem_balanco = [linha.strip() for linha in f.readlines()]

    # Filtrar papéis que possuem balanços
    df_empresas_filtrado = df_empresas[~df_empresas['Papel'].isin(sem_balanco)]

    # Criar um dicionário para mapear nomes de arquivos para papéis
    papel_map = {f"{row['Papel']}.zip": row['Papel'] for _, row in df_empresas_filtrado.iterrows()}

    # Iterar sobre todos os arquivos na pasta de downloads
    for nome_arquivo in os.listdir(download_path):
        caminho_arquivo_zip = os.path.join(download_path, nome_arquivo)

        if nome_arquivo.endswith(".zip") and nome_arquivo in papel_map:
            nome_papel = papel_map[nome_arquivo]
            try:
                with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
                    zip_ref.extractall(balancos_path)

                caminho_arquivo_xls = os.path.join(balancos_path, "balanco.xls")
                novo_nome_arquivo = os.path.join(balancos_path, f"{nome_papel}.xls")

                # Renomeia o arquivo se ainda não foi renomeado
                if os.path.exists(caminho_arquivo_xls):
                    os.rename(caminho_arquivo_xls, novo_nome_arquivo)
                    print(f"Arquivo {novo_nome_arquivo} extraído e renomeado com sucesso!")
                else:
                    print(f"Arquivo balanco.xls não encontrado em {balancos_path}")

            except zipfile.BadZipFile:
                print(f'O arquivo "{caminho_arquivo_zip}" não é um arquivo zip válido.')
            except Exception as e:
                logging.error(f"Erro ao extrair e renomear arquivo para {nome_papel}: {e}")

Arquivo balancos\AALR3.xls extraído e renomeado com sucesso!
Arquivo balancos\ABCB3.xls extraído e renomeado com sucesso!
Arquivo balancos\ABCB4.xls extraído e renomeado com sucesso!
Arquivo balancos\ABEV3.xls extraído e renomeado com sucesso!
Arquivo balancos\ABYA3.xls extraído e renomeado com sucesso!
Arquivo balancos\ACES3.xls extraído e renomeado com sucesso!
Arquivo balancos\ACES4.xls extraído e renomeado com sucesso!
Arquivo balancos\ADHM3.xls extraído e renomeado com sucesso!
Arquivo balancos\AEDU11.xls extraído e renomeado com sucesso!
Arquivo balancos\AEDU3.xls extraído e renomeado com sucesso!
Arquivo balancos\AELP3.xls extraído e renomeado com sucesso!
Arquivo balancos\AERI3.xls extraído e renomeado com sucesso!
Arquivo balancos\AESB3.xls extraído e renomeado com sucesso!
Arquivo balancos\AESL3.xls extraído e renomeado com sucesso!
Arquivo balancos\AESL4.xls extraído e renomeado com sucesso!
Arquivo balancos\AFLT3.xls extraído e renomeado com sucesso!
Arquivo balancos\AFLU3.

#### Chamando a função para extraír e renomear os arquivos

In [11]:
# chamada da função com o DataFrame de papéis
extrair_e_renomear_arquivo(df_empresas, DOWNLOADS_FOLDER, BALANCOS_FOLDER, SEM_BALANCOS_LISTA)

### Exibindo os resultados

#### Comparação entre dataframes

In [19]:
# Ler papéis do arquivo sem_balanco
sem_balanco_papeis = set()
if os.path.exists(SEM_BALANCOS_LISTA):
    with open(SEM_BALANCOS_LISTA, 'r') as f:
        sem_balanco_papeis = set(line.strip() for line in f.readlines())
else:
    print(f"O arquivo {SEM_BALANCOS_LISTA} não foi encontrado.")

# Obter as listas de papéis dos DataFrames e do arquivo 'sem_balancos'
df_empresas_papeis = set(df_empresas['Papel'])
df_financas_papeis = set(df_financas['Papel'])

# Papéis faltando em cada um dos conjuntos
faltando_empresas = df_financas_papeis - df_empresas_papeis
faltando_financas = df_empresas_papeis - df_financas_papeis
faltando_sem_balanco = (df_empresas_papeis | df_financas_papeis) - sem_balanco_papeis

# Papéis sobrando em cada um dos conjuntos
sobrando_empresas = df_empresas_papeis - df_financas_papeis
sobrando_financas = df_financas_papeis - df_empresas_papeis
sobrando_sem_balanco = sem_balanco_papeis - (df_empresas_papeis | df_financas_papeis)

# Papéis presentes em 'df_empresas' e 'df_financas', mas não em 'sem_balanco'
presentes_empresas_financas_nao_balanco = (df_empresas_papeis & df_financas_papeis) - sem_balanco_papeis

# Resultados
print("Papéis faltando em 'df_empresas':", faltando_empresas)
print("Papéis faltando em 'df_financas':", faltando_financas)


print("\nPapéis sobrando em 'df_empresas':", sobrando_empresas)
print("Papéis sobrando em 'df_financas':", sobrando_financas)


print("\nPapéis presentes em 'df_empresas' e 'df_financas', mas não em 'sem_balanco':", presentes_empresas_financas_nao_balanco)



NameError: name 'df_financas' is not defined